## Setup

### Import TensorFlow and other libraries

In [4]:
import tensorflow as tf
from tensorflow.keras.layers.experimental import preprocessing

import numpy as np
import os
import time

### Download the Shakespeare dataset

Change the following line to run this code on your own data.

In [ ]:
!git clone https://github.com/fwuffyboi/PROJECT-FRNN
path_to_file = "PROJECT-FRNN/content_got/smolFileContent" # tf.keras.utils.get_file('smolFileContent.txt', 'https://github.com/fwuffyboi/PROJECT-FRNN/blob/main/content_got/smolFileContent')

### Read the data

First, look in the text:

In [ ]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print(f'Length of text: {len(text)} characters')

FileNotFoundError: ignored

In [ ]:
# Take a look at the first 250 characters in text
print(text)

In [ ]:
# The unique characters in the file
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')

376 unique characters


## Process the text

### Vectorize the text

Before training, you need to convert the strings to a numerical representation. 

The `preprocessing.StringLookup` layer can convert each character into a numeric ID. It just needs the text to be split into tokens first.

In [ ]:
example_texts = ['abcdefg', 'xyz']

chars = tf.strings.unicode_split(example_texts, input_encoding='UTF-8')
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

Now create the `preprocessing.StringLookup` layer:

In [ ]:
ids_from_chars = preprocessing.StringLookup(
    vocabulary=list(vocab), mask_token=None)

It converts form tokens to character IDs:

In [ ]:
ids = ids_from_chars(chars)
ids

<tf.RaggedTensor [[68, 69, 70, 71, 72, 73, 74], [91, 92, 93]]>

Since the goal of this tutorial is to generate text, it will also be important to invert this representation and recover human-readable strings from it. For this you can use `preprocessing.StringLookup(..., invert=True)`.  

Note: Here instead of passing the original vocabulary generated with `sorted(set(text))` use the `get_vocabulary()` method of the `preprocessing.StringLookup` layer so that the `[UNK]` tokens is set the same way.

In [ ]:
chars_from_ids = tf.keras.layers.experimental.preprocessing.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)

This layer recovers the characters from the vectors of IDs, and returns them as a `tf.RaggedTensor` of characters:

In [ ]:
chars = chars_from_ids(ids)
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

You can `tf.strings.reduce_join` to join the characters back into strings. 

In [ ]:
tf.strings.reduce_join(chars, axis=-1).numpy()

array([b'abcdefg', b'xyz'], dtype=object)

In [ ]:
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

### The prediction task

Given a character, or a sequence of characters, what is the most probable next character? This is the task you're training the model to perform. The input to the model will be a sequence of characters, and you train the model to predict the output—the following character at each time step.

Since RNNs maintain an internal state that depends on the previously seen elements, given all the characters computed until this moment, what is the next character?


### Create training examples and targets

Next divide the text into example sequences. Each input sequence will contain `seq_length` characters from the text.

For each input sequence, the corresponding targets contain the same length of text, except shifted one character to the right.

So break the text into chunks of `seq_length+1`. For example, say `seq_length` is 4 and our text is "Hello". The input sequence would be "Hell", and the target sequence "ello".

To do this first use the `tf.data.Dataset.from_tensor_slices` function to convert the text vector into a stream of character indices.

In [ ]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

<tf.Tensor: shape=(83082,), dtype=int64, numpy=array([36, 86,  3, ..., 17,  2,  1])>

In [ ]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [ ]:
for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))

A
s
 
a
 
l
i
f
e
 


In [ ]:
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

The `batch` method lets you easily convert these individual characters to sequences of the desired size.

In [ ]:
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(1):
  print(chars_from_ids(seq))

tf.Tensor(
[b'A' b's' b' ' b'a' b' ' b'l' b'i' b'f' b'e' b' ' b'l' b'o' b'n' b'g'
 b' ' b'C' b'a' b'r' b'd' b'i' b'n' b'a' b'l' b's' b' ' b'f' b'a' b'n'
 b',' b' ' b'i' b't' b' ' b'h' b'a' b's' b' ' b'b' b'e' b'e' b'n' b' '
 b'f' b'u' b'n' b' ' b'f' b'o' b'l' b'l' b'o' b'w' b'i' b'n' b'g' b' '
 b't' b'h' b'i' b's' b' ' b'p' b'u' b'r' b's' b'u' b'i' b't' b' ' b'o'
 b'f' b' ' b'y' b'o' b'u' b'r' b' ' b't' b'a' b'l' b'e' b'n' b't' b's'
 b'.' b' ' b'G' b'o' b'o' b'd' b' ' b'l' b'u' b'c' b'k' b' ' b's' b'i'
 b'r' b'.' b' '], shape=(101,), dtype=string)


It's easier to see what this is doing if you join the tokens back into strings:

In [ ]:
for seq in sequences.take(5):
  print(text_from_ids(seq).numpy())

b'As a life long Cardinals fan, it has been fun following this pursuit of your talents. Good luck sir. '
b'I wish you the best.\r\n\xec\xb6\x94\xec\x9a\xb4\xec\x95\x84\xec\xb9\xa8\xec\x9d\xb4\xeb\x84\xa4\xec\x9a\xa5 \r\nI\xe2\x80\x99m sure they never will indeed. Anyway, what\xe2\x80\x99s the craic with that p'
b'hoto being taken at Green Dragon Lane? Did someone follow it from the LTN in hot pursuit to take a ph'
b"oto?\r\n\xed\x95\x98\xed\x95\x98\xed\x95\xb3\r\nTaxi Driver, It's OK that's Love, Ruler, Chip In (6 episode doang), The Veil, DP (Deserter"
b' Pursuit), Designated Survivor.\r\n\r\nItu tp bukan underrated, Nder. Bagus semua, tp barangkali kmu blm '


For training you'll need a dataset of `(input, label)` pairs. Where `input` and 
`label` are sequences. At each time step the input is the current character and the label is the next character. 

Here's a function that takes a sequence as input, duplicates, and shifts it to align the input and label for each timestep:

In [ ]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [ ]:
split_input_target(list("Tensorflow"))

(['T', 'e', 'n', 's', 'o', 'r', 'f', 'l', 'o'],
 ['e', 'n', 's', 'o', 'r', 'f', 'l', 'o', 'w'])

In [ ]:
dataset = sequences.map(split_input_target)

In [ ]:
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b'As a life long Cardinals fan, it has been fun following this pursuit of your talents. Good luck sir.'
Target: b's a life long Cardinals fan, it has been fun following this pursuit of your talents. Good luck sir. '


### Create training batches

You used `tf.data` to split the text into manageable sequences. But before feeding this data into the model, you need to shuffle the data and pack it into batches.

In [ ]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<PrefetchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

## Build The Model

This section defines the model as a `keras.Model` subclass (For details see [Making new Layers and Models via subclassing](https://www.tensorflow.org/guide/keras/custom_layers_and_models)). 

This model has three layers:

* `tf.keras.layers.Embedding`: The input layer. A trainable lookup table that will map each character-ID to a vector with `embedding_dim` dimensions;
* `tf.keras.layers.GRU`: A type of RNN with size `units=rnn_units` (You can also use an LSTM layer here.)
* `tf.keras.layers.Dense`: The output layer, with `vocab_size` outputs. It outputs one logit for each character in the vocabulary. These are the log-likelihood of each character according to the model.

In [ ]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [ ]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

In [ ]:
model = MyModel(
    # Be sure the vocabulary size matches the `StringLookup` layers.
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

For each character the model looks up the embedding, runs the GRU one timestep with the embedding as input, and applies the dense layer to generate logits predicting the log-likelihood of the next character:

![A drawing of the data passing through the model](https://github.com/tensorflow/text/blob/master/docs/tutorials/images/text_generation_training.png?raw=1)

Note: For training you could use a `keras.Sequential` model here. To  generate text later you'll need to manage the RNN's internal state. It's simpler to include the state input and output options upfront, than it is to rearrange the model architecture later. For more details see the [Keras RNN guide](https://www.tensorflow.org/guide/keras/rnn#rnn_state_reuse).

## Try the model

Now run the model to see that it behaves as expected.

First check the shape of the output:

In [ ]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 377) # (batch_size, sequence_length, vocab_size)


In the above example the sequence length of the input is `100` but the model can be run on inputs of any length:

In [ ]:
model.summary()

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  96512     
                                                                 
 gru (GRU)                   multiple                  3938304   
                                                                 
 dense (Dense)               multiple                  386425    
                                                                 
Total params: 4,421,241
Trainable params: 4,421,241
Non-trainable params: 0
_________________________________________________________________


To get actual predictions from the model you need to sample from the output distribution, to get actual character indices. This distribution is defined by the logits over the character vocabulary.

Note: It is important to _sample_ from this distribution as taking the _argmax_ of the distribution can easily get the model stuck in a loop.

Try it for the first example in the batch:

In [ ]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

This gives us, at each timestep, a prediction of the next character index:

In [ ]:
sampled_indices

array([306, 155, 166, 374,  78, 292,  80,  59, 211, 178,  58,  64, 363,
       137,  99, 365, 172, 251,  62, 174, 124, 236, 147,  83, 158, 366,
        36, 359, 374,  99, 224, 305, 153, 140, 247, 104, 292, 329, 132,
        24, 246, 306, 281, 291, 227, 341, 114, 121,  95,  15, 320, 128,
       304, 125,  59, 338,  90, 188,  61, 184,  91,  66, 102,  64, 152,
       276, 123, 367,  87, 179, 235, 328, 305, 128, 180,  55,  91, 259,
       245,  18, 299,   0, 331, 305,  44,  53, 147, 321, 195, 362,  38,
       203,  75,  45, 192, 227, 348, 122, 180, 226])

Decode these to see the text predicted by this untrained model:

In [ ]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b"ey're all pretty good. if ur makin the money to afford art of ur dogself getting plowed by ur husban"

Next Char Predictions:
 b'\xe7\xa9\xba\xe3\x81\x88\xe3\x81\x96\xf0\x9d\x92\xaak\xe6\xb1\x82mX\xe3\x82\xa3\xe3\x81\xa6W]\xef\xbc\x81\xe2\x96\xaa\xc2\xaf\xef\xbc\x9f\xe3\x81\x9c\xe4\xbd\x95[\xe3\x81\xa0\xe2\x80\x93\xe3\x83\xa9\xe3\x80\x81p\xe3\x81\x8c\xef\xbd\xa1A\xed\x95\x98\xf0\x9d\x92\xaa\xc2\xaf\xe3\x82\xbf\xe7\xa9\xb6\xe3\x81\x84\xe2\x98\x85\xe4\xba\xba\xc3\x89\xe6\xb1\x82\xeb\x8b\x9d\xe2\x82\x835\xe4\xba\x8b\xe7\xa9\xba\xe6\x9b\x9c\xe6\xb1\x81\xe3\x83\x8a\xec\x95\x88\xc3\xb1\xcb\x96},\xe9\x80\xb2\xe2\x80\x9a\xe7\xa9\x8d\xe2\x80\x94X\xeb\xad\x90w\xe3\x81\xb1Z\xe3\x81\xadx_\xc2\xb4]\xe3\x81\x83\xe6\x84\x9b\xcf\x89\xf0\x9d\x90\xb5t\xe3\x81\xa7\xe3\x83\xa5\xeb\x84\xa4\xe7\xa9\xb6\xe2\x80\x9a\xe3\x81\xa8Tx\xe5\x9e\xa2\xe4\xb8\x8a/\xe7\x90\x86[UNK]\xeb\x93\xa4\xe7\xa9\xb6IR\xe3\x80\x81\xe9\x81\x93\xe3\x81\xbf\xef\xb8\x8eC\xe3\x82\x89hJ\xe3\x81\xb6\xe3\x83\x8a\xec\x9a\xb4\xcc\x

## Train the model

At this point the problem can be treated as a standard classification problem. Given the previous RNN state, and the input this time step, predict the class of the next character.

### Attach an optimizer, and a loss function

The standard `tf.keras.losses.sparse_categorical_crossentropy` loss function works in this case because it is applied across the last dimension of the predictions.

Because your model returns logits, you need to set the `from_logits` flag.


In [ ]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [ ]:
example_batch_loss = loss(target_example_batch, example_batch_predictions)
mean_loss = example_batch_loss.numpy().mean()
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", mean_loss)

Prediction shape:  (64, 100, 377)  # (batch_size, sequence_length, vocab_size)
Mean loss:         5.9313817


A newly initialized model shouldn't be too sure of itself, the output logits should all have similar magnitudes. To confirm this you can check that the exponential of the mean loss is approximately equal to the vocabulary size. A much higher loss means the model is sure of its wrong answers, and is badly initialized:

In [ ]:
tf.exp(mean_loss).numpy()

376.67462

Configure the training procedure using the `tf.keras.Model.compile` method. Use `tf.keras.optimizers.Adam` with default arguments and the loss function.

In [ ]:
model.compile(optimizer='adam', loss=loss)

### Configure checkpoints

Use a `tf.keras.callbacks.ModelCheckpoint` to ensure that checkpoints are saved during training:

In [ ]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

### Execute the training

To keep training time reasonable, use 10 epochs to train the model. In Colab, set the runtime to GPU for faster training.

In [ ]:
EPOCHS = 300

In [ ]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

## Generate text

The simplest way to generate text with this model is to run it in a loop, and keep track of the model's internal state as you execute it.

![To generate text the model's output is fed back to the input](https://github.com/tensorflow/text/blob/master/docs/tutorials/images/text_generation_sampling.png?raw=1)

Each time you call the model you pass in some text and an internal state. The model returns a prediction for the next character and its new state. Pass the prediction and state back in to continue generating text.


The following makes a single step prediction:

In [ ]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [ ]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

Run it in a loop to generate some text. Looking at the generated text, you'll see the model knows when to capitalize, make paragraphs and imitates a Shakespeare-like writing vocabulary. With the small number of training epochs, it has not yet learned to form coherent sentences.

In [ ]:
start = time.time()
states = None
next_char = tf.constant(['ROMEO:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

ROMEO: Cowrres here so many furries on my flight. Even 
@JacentRabbit
 and I exdeal, what day ats a furry in o IMvar A amazen dorn that daybas staffing a duspack. Please don’t leave me babe #psycho
Every Lisages it male sthen #aniy #furry #furryart #furryancher #velpiop
 art sheer in a dryward " and it your doing tanka day affers even if you say their children don't have the same rights as the loako s  want aro geople dead furry pares it beasthal of excepoden in a qucter contrations here: http://ow.ly/Gq7S30s2BLw ^ajb

 Masks mandatory indoors
 #ReconnectingBC
Why are you looking for furries
My god Christian furries
IT What makes you like that implemented the nards of Life, liberty, and the pursuit of happiness to protect those rights of the leastor colldare, they whre were porn. The world et collegget that same-reisesen bity have to promet  urtoloting this recation by sonditions here: http://ow.ly/io3z30s2Ca2 ^ajb
 Masks mandatory indoors
 #ReconnectingBC
Really 9 wher u can pa 

____

The easiest thing you can do to improve the results is to train it for longer (try `EPOCHS = 30`).

You can also experiment with a different start string, try adding another RNN layer to improve the model's accuracy, or adjust the temperature parameter to generate more or less random predictions.

If you want the model to generate text *faster* the easiest thing you can do is batch the text generation. In the example below the model generates 5 outputs in about the same time it took to generate 1 above. 

In [ ]:
start = time.time()
states = None
next_char = tf.constant(['ROMEO:', 'ROMEO:', 'ROMEO:', 'ROMEO:', 'ROMEO:', 'ROMEO:', 'ROMEO:', 'ROMEO:', 'ROMEO:', 'ROMEO:', 'ROMEO:'])

result = [next_char]

for n in range(256):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result, '\n\n' + '_'*80)
print('\nRun time:', end - start)

tf.Tensor(
[b'ROMEO: Corts into weird shit.\r\nA har arraons of I draw furries but I know recently in the community and with tiktok, Zoophila(?) is how more chance on am*zo narda es that that are milled may cuman pursuit of academic success. \r\n@HHSBears\r\n  @NJASL\r\n @TDeeer, Cha'
 b"ROMEO: CREAT\r\n ARERG TOG: The Quer you ports, but I drew and erased that snout about 100 times. If I can make something 1000 times better.\r\n\xe3\x81\xa0\xe3\x81\x84\xe3\x81\xb6\xe8\x89\xaf\xe3\x81\x8f\xe3\x81\xaa\xe3\x81\xa3\xe3\x81\x9f\xe3\x82\x88\xef\xbc\x81\xe3\x82\x82\xe3\x81\x86\xe3\x81\xa1\xe3\x82\x87\xe3\x81\x84\xe3\x81\xa3\xe3\x81\xa6\xe6\x84\x9f\xe3\x81\x98( \xe3\x83\xbb\xe2\x88\x87\xe3\x83\xbb)\r\nCool\r\nI mean the R of 9 was without mitigation no? I'm just trying to say it's not doom and glo"
 b'ROMEO: DICK IN MY TIMELINE SO I\xe2\x80\x99M GOIN THROUGH SHIT\r\n\r\nIM GETTING TO THE BOTTOM OF THESE OPEN BOTTOMS HERE AND NOW\r\n\r\nFINNA SIFT THROUGH YALL BITCHES LIKE SOUP\r\nCara s\xc3\x

## Export the generator

This single-step model can easily be [saved and restored](https://www.tensorflow.org/guide/saved_model), allowing you to use it anywhere a `tf.saved_model` is accepted.

In [ ]:
tf.saved_model.save(one_step_model, 'one_step')
one_step_reloaded = tf.saved_model.load('one_step')

INFO:tensorflow:Assets written to: one_step/assets


INFO:tensorflow:Assets written to: one_step/assets


In [ ]:
states = None
next_char = tf.constant()
result = [next_char]

for n in range(100):
  next_char, states = one_step_reloaded.generate_one_step(next_char, states=states)
  result.append(next_char)

print(tf.strings.join(result)[0].numpy().decode("utf-8"))

TypeError: ignored

### Advanced: Customized Training

The above training procedure is simple, but does not give you much control.
It uses teacher-forcing which prevents bad predictions from being fed back to the model, so the model never learns to recover from mistakes.

So now that you've seen how to run the model manually next you'll implement the training loop. This gives a starting point if, for example, you want to implement _curriculum  learning_ to help stabilize the model's open-loop output.

The most important part of a custom training loop is the train step function.

Use `tf.GradientTape` to track the gradients. You can learn more about this approach by reading the [eager execution guide](https://www.tensorflow.org/guide/eager).

The basic procedure is:

1. Execute the model and calculate the loss under a `tf.GradientTape`.
2. Calculate the updates and apply them to the model using the optimizer.

In [ ]:
class CustomTraining(MyModel):
  @tf.function
  def train_step(self, inputs):
      inputs, labels = inputs
      with tf.GradientTape() as tape:
          predictions = self(inputs, training=True)
          loss = self.loss(labels, predictions)
      grads = tape.gradient(loss, model.trainable_variables)
      self.optimizer.apply_gradients(zip(grads, model.trainable_variables))

      return {'loss': loss}

The above implementation of the `train_step` method follows [Keras' `train_step` conventions](https://www.tensorflow.org/guide/keras/customizing_what_happens_in_fit). This is optional, but it allows you to change the behavior of the train step and still use keras' `Model.compile` and `Model.fit` methods.

In [ ]:
model = CustomTraining(
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [ ]:
model.compile(optimizer = tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True))

In [ ]:
model.fit(dataset, epochs=1)

Or if you need more control, you can write your own complete custom training loop:

In [ ]:
EPOCHS = 10

mean = tf.metrics.Mean()

for epoch in range(EPOCHS):
    start = time.time()

    mean.reset_states()
    for (batch_n, (inp, target)) in enumerate(dataset):
        logs = model.train_step([inp, target])
        mean.update_state(logs['loss'])

        if batch_n % 50 == 0:
            template = f"Epoch {epoch+1} Batch {batch_n} Loss {logs['loss']:.4f}"
            print(template)

    # saving (checkpoint) the model every 5 epochs
    if (epoch + 1) % 5 == 0:
        model.save_weights(checkpoint_prefix.format(epoch=epoch))

    print()
    print(f'Epoch {epoch+1} Loss: {mean.result().numpy():.4f}')
    print(f'Time taken for 1 epoch {time.time() - start:.2f} sec')
    print("_"*80)

model.save_weights(checkpoint_prefix.format(epoch=epoch))